In [1]:
import pandas as pd

C:\Users\qls05\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
df = pd.read_csv(r"C:\Users\qls05\OneDrive\바탕 화면\df.csv", encoding = 'cp949')

df['preprocessed_송출내용'] = df['preprocessed_송출내용'].fillna('')
texts = df['preprocessed_송출내용']

### 주어진 df ->  tf-idf 벡터화

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=30)
vectored_df = vectorizer.fit_transform(texts)

In [4]:
dense_df = vectored_df.todense() #vectored_df는 희소행렬이기 때문에 dense 형태로 전환.

feature_names = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(dense_df, columns=feature_names)

### 데이터 분할

In [5]:
from sklearn.model_selection import train_test_split

X = df_tfidf
y = df['label']

# train : val : test = 6 : 2 : 2
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) 


print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape, "y_valid shape:", y_valid.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (5608, 30) y_train shape: (5608,)
X_valid shape: (1870, 30) y_valid shape: (1870,)
X_test shape: (1870, 30) y_test shape: (1870,)


# ----------------------------------------------------------------------------

## RandomForest 모델 적용

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report

rf = RandomForestClassifier().fit(X_train, y_train)

y_valid_hat = dt.predict(X_valid)
valid_accuracy = accuracy_score(y_valid, y_valid_hat)

print("valid score: %.3f" %valid_accuracy)

valid score: 0.696


In [13]:
#상세 보고서
print(metrics.classification_report(y_valid, y_valid_hat))

              precision    recall  f1-score   support

           0       0.88      0.69      0.77       152
           1       0.55      0.72      0.62       162
           2       0.65      0.40      0.50        89
           3       0.58      0.27      0.37        51
           4       0.61      0.71      0.66       607
           5       0.80      0.85      0.82       464
           6       0.79      0.84      0.81       194
           7       0.80      0.29      0.43       151

    accuracy                           0.70      1870
   macro avg       0.71      0.60      0.62      1870
weighted avg       0.71      0.70      0.69      1870



## hyperparmeter 튜닝

**1. gird search로 조정**

In [14]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

grid_search.fit(X_train, y_train)

# 최적의 파라미터와 최고의 점수 출력
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


C:\Users\qls05\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
302 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\qls05\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\qls05\anaconda3\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "C:\Users\qls05\anaconda3\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\qls05\anaconda3\lib\site-packa

Best parameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Best score: 0.8405858978089654


## valid score

In [28]:
rf = RandomForestClassifier(max_depth= 20, max_features= 'log2', min_samples_leaf= 1,
                            min_samples_split= 10, n_estimators= 300).fit(X_train, y_train)

y_train_hat = rf.predict(X_train)
y_valid_hat =rf.predict(X_valid)
        
train_accuracy = accuracy_score(y_train, y_train_hat)
valid_accuracy = accuracy_score(y_valid, y_valid_hat)

print("train score: %.3f" %train_accuracy)
print("valid score: %.3f" %valid_accuracy)

train score: 0.903
valid score: 0.829


In [26]:
print(metrics.classification_report(y_valid, y_valid_hat))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91       152
           1       0.71      0.72      0.72       162
           2       0.72      0.55      0.62        89
           3       0.74      0.49      0.59        51
           4       0.77      0.87      0.82       607
           5       0.97      0.92      0.94       464
           6       0.79      0.87      0.83       194
           7       0.88      0.70      0.78       151

    accuracy                           0.83      1870
   macro avg       0.81      0.75      0.78      1870
weighted avg       0.83      0.83      0.83      1870

